In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from tensorflow import keras
from tensorflow.keras import layers
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, LSTM,Input,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Model
word2vec = None

Usamos los pre entrenados de word2vec

In [2]:
# bajado desde https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
# si tira error de ram ejecutar como root 'echo 1 > /proc/sys/vm/overcommit_memory'
embeddings_path = './data/embeddings/word2vec.bin'
if word2vec is None:
        word2vec = KeyedVectors.load_word2vec_format(embeddings_path, binary=True)

Spliteo para obtener vectores de train y test

In [4]:
max_len=300
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
X = train_df['text']
Y = train_df['target']
X_train,X_test,Y_train,Y_test = train_test_split(X.values,Y.values,test_size=0.33,random_state = 17)

Tokenizo y armo un mapa de index -> word

In [1]:
# Se deberian filtrar algunas stop words?
# revisar si se puede usar gensim.utils.simple_preprocess(line)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
# Switcheo para obtener index -> word
index_words= dict(map(lambda x: (x[1], x[0]),tokenizer.word_index.items()))
encoded_texts = tokenizer.texts_to_sequences(X_train)

NameError: name 'Tokenizer' is not defined

Armo cada texto como un vector que tiene el promedio de los pesos de todas las palabras que componen a ese texto

In [2]:
embedding_texts = []
for text_encoded in encoded_texts:
    avg_text_embedding = np.zeros(max_len)
    for word_encoded in text_encoded:
        word = index_words[word_encoded]
        count = 0
        if word in word2vec:
            avg_text_embedding += word2vec[index_words[word_encoded]]
            count += 1
    avg_text = avg_text_embedding / count if count > 0 else avg_text_embedding
    embedding_texts.append(avg_text)

NameError: name 'encoded_texts' is not defined

Tengo que pasar esa lista de arrays a matriz y despues hacer un reshaped para que sea un arreglo 3D, esto es necesario para luego aplicarlo al modelo sin necesidad de hacer un Embedding

In [ ]:
matrix = np.vstack(embedding_texts)
X_train_reshaped = matrix.reshape(matrix.shape[0], 1, matrix.shape[1])

Armo el RNN sin el Embedding, pero le pongo el input shape

In [ ]:
def RNN():
    model = Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)])
    return model

Compilo ni modelo

In [ ]:
model = RNN()
model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Entrenamos

In [ ]:
model.fit(X_train_reshaped, Y_train, epochs=20, shuffle=True, validation_split=0.2)

Importante: para poder evaluar el X test, tambien tenemos que tokenizarlo y armar los textos como vectores con el promedio de los pesos, esto tiene que ser asi porque tiene que entender como evaluar el twitter

In [ ]:
tokenizer2 = Tokenizer()
# Obtengo las palabras rankeadas
tokenizer2.fit_on_texts(X_test)
# Switcheo para obtener index -> word
index_words= dict(map(lambda x: (x[1], x[0]),tokenizer2.word_index.items()))
# Encodeo los valores
test_sequences = tokenizer2.texts_to_sequences(X_test)

embedding_texts_test = []
for text_encoded in test_sequences:
    avg_text_embedding = np.zeros(max_len)
    for word_encoded in text_encoded:
        word = index_words[word_encoded]
        count = 0
        if word in word2vec:
            avg_text_embedding += word2vec[index_words[word_encoded]]
            count += 1
    avg_text = avg_text_embedding / count if count > 0 else avg_text_embedding
    embedding_texts_test.append(avg_text)
    
matrix = np.vstack(embedding_texts_test)
X_test_reshaped = matrix.reshape(matrix.shape[0], 1, matrix.shape[1])

In [ ]:
accr = model.evaluate(X_test_reshaped,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))